In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
games = pd.read_csv("game_table.txt")
states = pd.read_csv("State_table.txt", usecols=['state_id', 'state', 'game_id'])

In [3]:
games

,game_id,result
0,1,0
1,2,0
2,3,1
3,4,1
4,5,1
...,...,...
47200,47201,0
47201,47202,0
47202,47203,1
47203,47204,1


In [4]:
states

,state_id,state,game_id
0,1,b:b/bb/bbb/bbbb/3/4/3/4/3/wwww/www/ww/w,1
1,2,w:b/bb/1bb/bbbb/b2/4/3/4/3/wwww/www/ww/w,1
2,3,b:b/bb/1bb/bbbb/b2/4/3/4/1w1/w1ww/www/ww/w,1
3,4,w:b/bb/1bb/bb1b/bb1/4/3/4/1w1/w1ww/www/ww/w,1
4,5,b:b/bb/1bb/bb1b/bb1/4/3/4/1ww/w1w1/www/ww/w,1
...,...,...,...
3122904,3122905,w:1/2/1w1/1ww1/3/2b1/bb1/4/3/4/3/b1/w,47205
3122905,3122906,b:1/1w/1w1/1w2/3/2b1/bb1/4/3/4/3/b1/w,47205
3122906,3122907,w:1/1w/1w1/1w2/3/4/bbb/4/3/4/3/b1/w,47205
3122907,3122908,b:1/ww/1w1/4/3/4/bbb/4/3/4/3/b1/w,47205


In [5]:
joined = states.merge(games, on="game_id")

In [6]:
joined

,state_id,state,game_id,result
0,1,b:b/bb/bbb/bbbb/3/4/3/4/3/wwww/www/ww/w,1,0
1,2,w:b/bb/1bb/bbbb/b2/4/3/4/3/wwww/www/ww/w,1,0
2,3,b:b/bb/1bb/bbbb/b2/4/3/4/1w1/w1ww/www/ww/w,1,0
3,4,w:b/bb/1bb/bb1b/bb1/4/3/4/1w1/w1ww/www/ww/w,1,0
4,5,b:b/bb/1bb/bb1b/bb1/4/3/4/1ww/w1w1/www/ww/w,1,0
...,...,...,...,...
3122904,3122905,w:1/2/1w1/1ww1/3/2b1/bb1/4/3/4/3/b1/w,47205,1
3122905,3122906,b:1/1w/1w1/1w2/3/2b1/bb1/4/3/4/3/b1/w,47205,1
3122906,3122907,w:1/1w/1w1/1w2/3/4/bbb/4/3/4/3/b1/w,47205,1
3122907,3122908,b:1/ww/1w1/4/3/4/bbb/4/3/4/3/b1/w,47205,1


In [7]:
grouped = joined.groupby(by=["state"], as_index=False).agg({"result": ['sum', 'count']})

In [8]:
grouped

state result      
                                                        sum count
0        b:1/1b/1b1/1b1b/1b1/4/1w1/4/2w/bw2/www/1w/w      1     1
1       b:1/1b/1b1/1b1b/1b1/4/1w1/4/bw1/2w1/bw1/ww/w      1     1
2         b:1/1b/1b1/1b1b/1w1/4/b2/1w2/3/2ww/2w/w1/w      2     2
3          b:1/1b/1b1/1b1b/1ww/4/b2/4/1w1/4/bw1/wb/w      1     1
4          b:1/1b/1b1/1b1b/2b/1b2/1w1/4/b2/4/2w/ww/w      1     1
...                                              ...    ...   ...
860982           w:b/ww/wbw/4/3/4/3/4/2w/b3/1b1/bb/w      0     2
860983           w:b/ww/wbw/4/3/4/3/4/3/1b2/bw1/bb/w      0     1
860984           w:b/ww/wbw/4/3/4/3/4/3/2w1/bb1/bb/w      0     1
860985          w:b/ww/wbw/4/3/4/3/4/3/b1w1/1bb/b1/w      0     2
860986           w:b/ww/wbw/4/3/4/3/4/3/bb2/1wb/b1/w      0     2

[860987 rows x 3 columns]

In [9]:
grouped['black_win_ratio'] = 1-grouped['result']['sum']/grouped['result']['count']

In [11]:
grouped

state result        \
                                                        sum count   
0        b:1/1b/1b1/1b1b/1b1/4/1w1/4/2w/bw2/www/1w/w      1     1   
1       b:1/1b/1b1/1b1b/1b1/4/1w1/4/bw1/2w1/bw1/ww/w      1     1   
2         b:1/1b/1b1/1b1b/1w1/4/b2/1w2/3/2ww/2w/w1/w      2     2   
3          b:1/1b/1b1/1b1b/1ww/4/b2/4/1w1/4/bw1/wb/w      1     1   
4          b:1/1b/1b1/1b1b/2b/1b2/1w1/4/b2/4/2w/ww/w      1     1   
...                                              ...    ...   ...   
860982           w:b/ww/wbw/4/3/4/3/4/2w/b3/1b1/bb/w      0     2   
860983           w:b/ww/wbw/4/3/4/3/4/3/1b2/bw1/bb/w      0     1   
860984           w:b/ww/wbw/4/3/4/3/4/3/2w1/bb1/bb/w      0     1   
860985          w:b/ww/wbw/4/3/4/3/4/3/b1w1/1bb/b1/w      0     2   
860986           w:b/ww/wbw/4/3/4/3/4/3/bb2/1wb/b1/w      0     2   

       black_win_ratio  
                        
0                  0.0  
1                  0.0  
2                  0.0  
3                  0.0  
4                  0.0  
...                ...  
860982             1.0  
860983             1.0  
860984             1.0  
860985             1.0  
860986             1.0  

[860987 rows x 4 columns]

In [51]:
def transform_state(state):
    board = []
    for i in state[2:]:
        if i == 'b':
            board.append(2)
        elif i == 'w':
            board.append(1)
        elif i == '/':
            continue
        else:
            for j in range(int(i)):
                board.append(0)
            
    return board

def split_array(arr):
    return pd.Series(arr)

In [52]:
transformed_states = grouped['state'].apply(transform_state)

In [53]:
new_cols = pd.DataFrame(transformed_states.tolist())

In [54]:
new_cols

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0,0,2,0,2,0,0,2,0,2,...,2,1,0,0,1,1,1,0,1,1
1,0,0,2,0,2,0,0,2,0,2,...,0,0,1,0,2,1,0,1,1,1
2,0,0,2,0,2,0,0,2,0,2,...,0,0,1,1,0,0,1,1,0,1
3,0,0,2,0,2,0,0,2,0,2,...,0,0,0,0,2,1,0,1,2,1
4,0,0,2,0,2,0,0,2,0,2,...,0,0,0,0,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860982,2,1,1,1,2,1,0,0,0,0,...,2,0,0,0,0,2,0,2,2,1
860983,2,1,1,1,2,1,0,0,0,0,...,0,2,0,0,2,1,0,2,2,1
860984,2,1,1,1,2,1,0,0,0,0,...,0,0,1,0,2,2,0,2,2,1
860985,2,1,1,1,2,1,0,0,0,0,...,2,0,1,0,0,2,2,2,0,1


In [24]:
import sklearn
from sklearn.neural_network import MLPRegressor

In [55]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(new_cols, grouped['black_win_ratio'])

In [56]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
857428,2,1,1,0,2,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
587914,0,1,0,0,0,0,0,0,2,0,...,2,0,0,0,1,1,0,0,1,0
471030,0,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
728706,2,2,0,0,0,2,0,0,0,1,...,0,0,1,0,2,0,0,1,1,1
515528,0,0,0,2,1,0,0,0,0,0,...,0,2,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32205,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
412266,2,1,2,0,1,0,0,0,0,0,...,0,0,2,0,0,0,2,1,1,1
418835,2,1,2,2,2,0,0,0,2,0,...,0,0,0,1,1,0,1,1,1,1
174027,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
regr = MLPRegressor().fit(X_train, y_train)

In [59]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, regr.predict(X_test))

0.08796996777262692

In [60]:
import joblib

In [61]:
joblib.dump(regr, 'nn.joblib')

['nn.joblib']